In [21]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd
import numpy as np
from ta.trend import PSARIndicator

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def trade_option(symbolID, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    start_time = start_time.replace(hour=9, minute=15, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)

    ourl = 'https://groww.in/v1/api/stocks_fo_data/v4/charting_service/chart/exchange/NSE/segment/FNO/'+symbolID
    url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+'NIFTY'
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(ourl, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    ohlc.drop(['high', 'low'], axis=1, inplace=True)
    ohlc['EMA_6'] = ohlc['close'].ewm(span=6, adjust=False).mean()
    return ohlc

In [25]:
symbol1 = 'BANKNIFTY2461249300CE'
symbol2 = 'NIFTY2461322700CE'
days_before =1
timeframe = 30
ohlc = trade_option(symbol2, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
stop_loss = 5
take_profit = 5
daily_pl =0
daily_pl_limit = 10

for i, row in ohlc.iterrows():
    if i == 0:
        continue

    if ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['open'] and ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['EMA_6']:
        if not position:
            position = True
            entry_price = row['open']
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['EMA_6'] or row['open'] <= (entry_price-stop_loss) or row['open'] >= (entry_price+take_profit):
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            daily_pl += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*25)-(trades_taken*60)}")

Long: Entry Time: 2024-06-07 10:15:00, Entry Price: 395.0
Exit Long: Time: 2024-06-07 10:45:00, Exit Price: 436.05, P/L: 41.05000000000001
Long: Entry Time: 2024-06-07 11:15:00, Entry Price: 470.15
Exit Long: Time: 2024-06-07 11:45:00, Exit Price: 525.65, P/L: 55.5
Long: Entry Time: 2024-06-07 12:15:00, Entry Price: 574.0
Exit Long: Time: 2024-06-07 12:45:00, Exit Price: 596.05, P/L: 22.049999999999955
Long: Entry Time: 2024-06-07 13:45:00, Entry Price: 602.55
Exit Long: Time: 2024-06-07 14:15:00, Exit Price: 618.5, P/L: 15.950000000000045
Long: Entry Time: 2024-06-07 15:15:00, Entry Price: 648.3

Trades count: 5 cost: 300

Total Profit/Loss: 134.6

Take Away: 3065.0


In [34]:
symbol1 = 'BANKNIFTY2461249300CE'
symbol2 = 'NIFTY2461322700CE'
days_before =14
timeframe = 60
ohlc = trade_option(symbol2, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
stop_loss = 5
take_profit = 5
daily_pl =0
daily_pl_limit = 50

for i, row in ohlc.iterrows():
    if i == 0:
        continue

    if ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['open'] and ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['EMA_6'] :
        if not position:
            position = True
            entry_price = row['open']
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['EMA_6'] or row['open'] <= (entry_price-stop_loss) or row['open'] >= (entry_price+take_profit):
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            daily_pl += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*25)-(trades_taken*60)}")

Long: Entry Time: 2024-05-27 11:34:00, Entry Price: 741.85
Exit Long: Time: 2024-05-27 12:35:00, Exit Price: 826.8, P/L: 84.94999999999993
Long: Entry Time: 2024-05-31 14:15:00, Entry Price: 477.3
Exit Long: Time: 2024-05-31 15:15:00, Exit Price: 466.0, P/L: -11.300000000000011
Long: Entry Time: 2024-06-03 09:15:00, Entry Price: 750.0
Exit Long: Time: 2024-06-03 10:15:00, Exit Price: 729.55, P/L: -20.450000000000045
Long: Entry Time: 2024-06-03 12:15:00, Entry Price: 750.55
Exit Long: Time: 2024-06-03 13:15:00, Exit Price: 775.0, P/L: 24.450000000000045
Long: Entry Time: 2024-06-03 14:15:00, Entry Price: 788.35
Exit Long: Time: 2024-06-03 15:15:00, Exit Price: 822.0, P/L: 33.64999999999998
Long: Entry Time: 2024-06-04 09:15:00, Entry Price: 832.2
Exit Long: Time: 2024-06-04 10:15:00, Exit Price: 537.05, P/L: -295.1500000000001
Long: Entry Time: 2024-06-05 14:15:00, Entry Price: 189.75
Exit Long: Time: 2024-06-05 15:15:00, Exit Price: 252.5, P/L: 62.75
Long: Entry Time: 2024-06-06 10:15